In [1]:
import pandas as pd
from pandas import read_csv
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.signal import medfilt
from scipy.signal import butter,filtfilt
from scipy.signal import iirfilter
import itertools
from scipy import fftpack
from scipy import signal
from skimage.util.shape import view_as_windows
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import glob
from scipy.fftpack import rfft,fftshift
from numpy.fft import fft,fftfreq
from scipy.integrate import trapz
from sklearn.model_selection import train_test_split

## Feature Extraction
def load_file(filepath):
    column_names = ['Ax','Ay','Az']
    dataframe = read_csv(filepath, header=None, names=column_names, delim_whitespace=True)
#     get_num_from_path = re.findall("\d",filepath)
    return dataframe

#Calculating the entropy - which is the the messure of the randomness of the data
def entropy(x):
    np_fft = np.fft.fft(x)
    amplitudes = 2 / len(x) * np.abs(np_fft)
    PSD = np.abs(amplitudes)**2/len(x)
    pi = PSD/np.sum(PSD)
    e = -np.sum(np.multiply(pi,np.log(pi)))
    return(e)
            
      

# IIR filter
# low pass filtering (LPF),where custom third-order elliptical infinite impulse
# response (IIR) filter with cut-off frequency at 0.25 Hz (0.01 dB passband ripple; stopband at −100 dB) is employed to
# separate the acceleration components due to gravity (GA) and bodily motion (BA) 
def IIR_lowpass_filter(data, cutoff, fs, rp, rs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = iirfilter(order, normal_cutoff, rp, rs, btype='low', analog=False, ftype='ellip')
    y = filtfilt(b, a, data)
    return y

#  butter filter
# A third order low-pass Butterworth filter with a cutoff frequency of 20 Hz.
#since the energy spectrum of the human body motion lies mainly within the range of 0 Hz to 15 Hz
def butter_lowpass_filter(data, cutoff,fs_in, order ,fs_out):
    nyq = 0.5 * fs_in
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False, fs=fs_out)
    y = filtfilt(b, a, data)
    return y

## total square root of sum of squers of the data
def signalMag(x,y,z):
    return np.sqrt(x*x + y*y + z*z)


### Calculating frequeancy parameter with DFT - Descrete Fourier Transform
## The transform component represent the frequencies with the higher amplitude in the signal

def freq_calc(y,index):
    n = len(y)
    freqs = fftfreq(n)
    mask = freqs > 0
    fft_val= fft(y,n)
    amplitude = 2*np.abs(fft_val/n)
    amplitude = amplitude[mask]
    max_six_index = np.argpartition(amplitude, -6)[-6:]
    max_six_ferq = freqs[mask][max_six_index]
    average_freq = np.average(freqs[mask],weights=amplitude)
    energy = np.mean(np.sum(amplitude**2))
    bandsEnergy = np.sum(amplitude**2)
    results = np.hstack([max_six_index, max_six_ferq, average_freq,energy,bandsEnergy])
    return results[index]

### Compute SMA - normalized signal magnitude area
def SMA_calac(header,features, data, window_size):
    data_abs = data.abs()
    data_inagration = pd.DataFrame()
    for col in data.columns:
        data_inagration[col] = data_abs[col].rolling(window_size).apply(trapz, raw=False)
    features[header+'_sma'] =  (1/2.56)*data_inagration.sum(axis = 1)
    return features 

### Utility function computing statistic features in time and frequency domain 
def time_freq_domain_feature_calc(header,y,window_size,features):
    y['total_Acc']  = signalMag(y['Ax'],y['Ay'],y['Az'])
    rolling_window = y.rolling(window_size)
    features[[header+'x_mean',header+'y_mean',header+'z_mean',header+'total_mean']] = rolling_window.mean()
    features[[header+'x_std',header+'y_std',header+'z_std',header+'total_std']] = rolling_window.std()
    features[[header+'x_median',header+'y_median',header+'z_median',header+'total_median']] = rolling_window.median()
    features[[header+'x_max',header+'y_max',header+'z_max',header+'total_max']] = rolling_window.max()
    features[[header+'x_min',header+'y_min',header+'z_min',header+'total_min']] = rolling_window.min()
    features[[header+'x_range',header+'y_range',header+'z_range',header+'total_range']] = rolling_window.max()-rolling_window.min()
    features[[header+'x_iqr',header+'y_iqr',header+'z_iqr',header+'total_iqr']] = rolling_window.quantile(0.75)-rolling_window.quantile(0.25)
    features[[header+'x_skew',header+'y_skew',header+'z_skew',header+'total_skew']] = rolling_window.skew()
    features[[header+'x_entropy',header+'y_entropy',header+'z_entropy',header+'total_entropy']] = rolling_window.apply(entropy, raw=True)
    ### Add SMA calac to features
    features[header+'_sma'] = SMA_calac(header,features, y[['Ax','Ay','Az']], window_size)
    ## calc rolling window for total compnent
    rolling_window_total = y['total_Acc'].rolling(window_size)
       
    #Calc freq features on the total component
    for i in np.arange(14):
        fet_temp = rolling_window_total.apply(lambda x: freq_calc(x,i))
        features[header+'freq_feature'+str(i)]=fet_temp[::]
        
        
    return features

# calculation the jerk - the derivative of the acceleration by (dA/dt)
def jerk_calc(df,dt):
    jerk_db=df.diff()/dt
    return jerk_db

#calculation cross correlation between Axes
def cross_correlation(features, data, window_size):
    corr = data.rolling(window_size).corr(pairwise=True)
    combo = list(itertools.combinations(data.columns,2))
    for i in np.arange(len(combo)):
        features[combo[i][0]+'_'+combo[i][1]+'_correleation']= corr[combo[i][0]].xs(combo[i][1],level=1)
    return features
        


## Main function for signal processing and feature extraction
def featuer_extraction(input_root,file_name, start, end, label):
    data = load_file(f"{input_root}/{file_name}")
    #Cut relevant part using end,start
    data = data.truncate(before=start-1, after=end-1)
    ### Apply Median filter with window=5 to reduce white noise (normal noise)
    TA_med_filter= pd.DataFrame({'Ax':medfilt(data.Ax,kernel_size= 3),
                                   'Ay':medfilt(data.Ay,kernel_size=3),
                                   'Az':medfilt(data.Az,kernel_size=3)})
    
    #plot_acc_together(data, TA_med_filter, 'Compare TA raw and with median Filter')
    ### A third order low-pass Butterworth filter with a cutoff frequency of 20 Hz.
    fs_in = 50       # sample rate of source data [Hz]
    cutoff = 20 # The Low-pass filter, passes signals with a frequency lower than a certain cutoff frequency and attenuates signals with frequencies higher than the cutoff frequency.
    order=3
    fs_out = 20
    TA_lowpass_filter = pd.DataFrame({'Ax':butter_lowpass_filter(TA_med_filter.Ax, cutoff, fs_in, order,fs_out),
                                   'Ay':butter_lowpass_filter(TA_med_filter.Ay, cutoff, fs_in, order,fs_out),
                                   'Az':butter_lowpass_filter(TA_med_filter.Az, cutoff, fs_in, order,fs_out)})
    #plot_acc_together(TA_med_filter, TA_lowpass_filter, 'Compare TA with median Filter and low pass filter')
    #### Gravity calculation
    fs= 50       # sample rate of source data [Hz]
    cutoff_gravity = 0.25 # The Low-pass filter, passes signals with a frequency lower than a 
                            #certain cutoff frequency and attenuates signals with frequencies higher than the cutoff frequency.
    order=3
    rp = 0.01 #passband ripple
    rs = 100 # stop band 
    #The Gravity Acceleration (GA) component is taken directly from the result of applying the LPF to the TA_lowpass_filter 
    Gravity = pd.DataFrame({'Ax':IIR_lowpass_filter(TA_lowpass_filter.Ax, cutoff_gravity, fs, rp, rs, order),
                               'Ay':IIR_lowpass_filter(TA_lowpass_filter.Ay, cutoff_gravity, fs, rp, rs, order),
                               'Az':IIR_lowpass_filter(TA_lowpass_filter.Az, cutoff_gravity, fs, rp, rs, order)})
    # Body Acceleraton (BA) component is taken as the difference between the original signal and the GA component
    
    BA_data= pd.DataFrame({'Ax':TA_lowpass_filter.Ax-Gravity.Ax,
                               'Ay':TA_lowpass_filter.Ay-Gravity.Ay,
                               'Az':TA_lowpass_filter.Az-Gravity.Az})
    
    #plot_acc_together(data_lowpass_filter, body_acceleration, 'Compare TA with BA filter')
    ## calculate jerk from BA
    dt = 1/50
    jerk_BA = jerk_calc(BA_data,dt)
    # Sloding window - each with a span of 2.56 sec and an overlap of 50% . This was determined according to 
    # values in literature, and is known to cpature human movment. 128 sample = 2.56 sec * 5-Htz sampling rate
    # overlaping - 64 samles
    window_size = 128
    features = pd.DataFrame()
    # TA = Total acceleration
    features = time_freq_domain_feature_calc('TA',TA_lowpass_filter,window_size,features)
    # BA = body acceleration
    features = time_freq_domain_feature_calc('BA',BA_data,window_size,features)
    # GA = Gravity acceleration
    features = time_freq_domain_feature_calc('GA',Gravity,window_size,features)
    # BJ = Body Jerk
    features = time_freq_domain_feature_calc('BJ',jerk_BA,window_size,features)  
    ### Compute cross correlation
    features = cross_correlation(features, BA_data, window_size)
    ### selecting only sliding window with 50% overlapping
    features = features[128::64]
    ### setting the labling of this set
    features['label']= np.ones(len(features))*label
    return features 



#### fumction that uploads the HARP dataset from input_root in local machone and creates train and test data
## Train and test files are saved to output root
## train, test - initialized dataframes. 
## partitian is made with 70% train in 30% test
#ind_test, ind_train - in case of partial run, can be used to save running time.
def load_dataset(input_root,train,test,ind_train,ind_test):
    files_list = glob.glob(input_root+'acc*.*')
    column_names = ['exp_id','user_id','act_id','label_start','label_end']
    labels = read_csv(input_root+'labels.txt', header=None, names=column_names, delim_whitespace=True)
    X = labels[['exp_id','user_id','label_start','label_end']] 
    Y = labels['act_id']
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
    #create_train_set
    X_train = X_train.reset_index()
    y_train = y_train.reset_index()
    train = pd.DataFrame()
    for i in np.arange(ind_train,len(X_train)):
        #Validatin lenght of the segment so i will not be less then 2.56sec.
        #Otherwise the segment is excluded from the data
        if X_train.label_end[i]-X_train.label_start[i]>=129: 
            file_name='acc_exp{:0>2d}_user{:0>2d}.txt'.format(X_train.exp_id[i],X_train.user_id[i],y_train.act_id[i])
            train = train.append(
                featuer_extraction(input_root,file_name, X_train.label_start[i], X_train.label_end[i],y_train.act_id[i]),
                ignore_index=True)
        else:
            print('sample_'+str(i)+'was excluded')
    print('finish_train')

    #create_test_set
    X_test = X_test.reset_index()
    y_test = y_test.reset_index()
    test = pd.DataFrame()
    for i in np.arange(ind_test,len(X_test)):
        #Validatin lenght of the segment so i will not be less then 2.56sec.
        #Otherwise the segment is excluded from the data
        if X_test.label_end[i]-X_test.label_start[i]>=129:
            file_name='acc_exp{:0>2d}_user{:0>2d}.txt'.format(X_test.exp_id[i],X_test.user_id[i],y_test.act_id[i])
            test = test.append(
                featuer_extraction(input_root,file_name, X_test.label_start[i], X_test.label_end[i],y_test.act_id[i]),
                ignore_index=True)
               
        else:
            print('sample_'+str(i)+'was excluded')
    print('finish_test')
    return train, test


## utility fimction to activate load_dataset function 
def extract_features_main(input_root,output_root):
    # insert ind_train = 780 and ind_test = 290 for a short run of the feature extraction module
    ind_train = 0
    ind_test = 0
    test = pd.DataFrame()
    train = pd.DataFrame()
    train,test = load_dataset(input_root,train,test,ind_train,ind_test)
    train.to_csv(output_root+'train_new.csv')
    test.to_csv(output_root+'test_new.csv')
    

## Main function for creating and training random forest model withe feature ranking and selection
def RandForest_model(input_path,selection_treshhold):
    ## excecute model with data saved in output folder
    ### load Train and split features and labels
    data_train = pd.read_csv( input_path+"train_new.csv")
    labels_train = data_train.label
    data_train=data_train.drop('label', axis=1)
    ### load Train and split features and labels
    data_test = pd.read_csv(input_path+"test_new.csv")
    labels_test = data_test.label
    data_test = data_test.drop('label', axis=1)
    print( "loading done")

    # Create the model with 1000 trees

    model = RandomForestClassifier(n_estimators=1000, 
                                   bootstrap = True,
                                   max_features = 'sqrt',
                                  criterion= 'entropy')

    rf = RandomForestClassifier(n_estimators=1000,random_state=0, n_jobs=-1)
    rf.fit(data_train, labels_train)
    predictions = rf.predict(data_test)
    ## report accuracy
    print('accuracy:' + str(np.sum(predictions == labels_test)/predictions.shape[0]))
    print(classification_report(labels_test, predictions, digits = 4))
    
    #Feture  scoring and selection
    feat_labels = data_train.columns
    for feature in zip(feat_labels, rf.feature_importances_):
        print(feature)
    
    ###select the features above gini score as determined in selection_treshhold
    selcted_features = SelectFromModel(rf, prefit=True, threshold=selection_treshhold)
    
    # Print the names of the most important features
    for feature_list_index in selcted_features.get_support(indices=True):
        print(feat_labels[feature_list_index])
    
    important_data_train = selcted_features.transform(data_train)
    important_data_test = selcted_features.transform(data_test)
    
    # Print the shape of the train dataframe before and after feature selection to varify it os changed
    print(important_data_train.shape, data_train.shape)
    
    ## train and estimate Random forest with selected features
    rf_important = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)
    rf_important.fit(important_data_train, labels_train)
    predictions_important = rf_important.predict(important_data_test)
    ## report accuracy
    print('accuracy:' + str(np.sum(predictions_important == labels_test)/predictions_important.shape[0]))
    print(classification_report(labels_test, predictions_important, digits = 4))
 
#This code allow control on executing the model. 
#inpur root - where HARP database is saved in folder
input_root = r'C:/Users/Yair/Documents/Airtasker/Akhila/HAPT Data Set/RawData/'
#Output root - where to store fetures . using the same path to lunch them for in RandForest_model 
output_root = r'C:\Users\Yair\Documents\Airtasker\Akhila\HAPT Data Set\Extracted_Features\\'
### Deactivate this line to avoid feature extracture
extract_features_main(input_root,output_root)
selection_treshhold = 0.05 #treshhold for gini ranking for fature selection
RandForest_model(output_root,selection_treshhold)

loading done
accuracy:0.9187519308001235
              precision    recall  f1-score   support

         1.0     0.9911    0.9875    0.9893       562
         2.0     0.9563    0.9850    0.9704       400
         3.0     0.9840    0.9608    0.9723       383
         4.0     0.8870    0.8416    0.8637       606
         5.0     0.8307    0.8826    0.8558       528
         6.0     0.9934    0.9983    0.9959       605
         7.0     0.6765    0.9200    0.7797        25
         8.0     1.0000    0.2000    0.3333        15
         9.0     0.7083    0.4359    0.5397        39
        10.0     0.1429    0.1111    0.1250        18
        11.0     0.4118    0.6000    0.4884        35
        12.0     0.5238    0.5238    0.5238        21

    accuracy                         0.9188      3237
   macro avg     0.7588    0.7039    0.7031      3237
weighted avg     0.9209    0.9188    0.9176      3237

('Unnamed: 0', 0.003145063539936353)
('TAx_mean', 0.01631386191172002)
('TAy_mean', 0.013979

C:\Users\Yair\Anaconda3\lib\site-packages\sklearn\feature_selection\base.py:79: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  UserWarning)
C:\Users\Yair\Anaconda3\lib\site-packages\sklearn\feature_selection\base.py:79: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  UserWarning)


(7668, 0) (7668, 211)


ValueError: Found array with 0 feature(s) (shape=(7668, 0)) while a minimum of 1 is required.

In [19]:
train.shapeddd

NameError: name 'train' is not defined